## Problem Statement 

The assignment of demands (2D items) to vehicle compartments (stacks) in order to be transported from sources to targets. <br>

##### Assumptions
- Every customer / vertex represents a single request. (A single item to be transported).
    - In case 2 items need to be picked from the same location, the model considers this as 2 pickup vertices (then the distance / cost can be initialized with 0).

#### Data 

- $G = (V, A)$
- $P = \{1, .., n\}$ $n$ pickup vertices for $n$ requests to be transported
- $I = \{(l_{i}, w_{i}), .. | \; i \; \in \; P\}$ $n$ 2D items representing the $n$ requests to be transported
- $D = \{n + 1, .., 2n\}$ $n$ delivery vertices for $n$ requests to be transported
- $V = \{0, 1, .., 2n\}$, where $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $M = \{1,.., m\}$ the set of Vehicle Compartments (stacks).
- $CompartmentDimensions = \{(l_{k}, d_{k}), .. |\; k \; \in \; M\}$
- $CompartmentCapacities = \{d_{k} |\; k \; \in \; M\}$ where $d_{k}$ is the *depth* of compartment $k$.

##### Inferred Data
$CompartmentCapabilities$ Matrix $MxI$
- $a_{k,\; i} = 1$ if item $i$ can be assigned to compartment $k$
- $a_{k,\; i} = 0$ otherwise
    
We say that $i$ is compatible with $k$ if it can fit in the compartment, considering both item orientations. In other words:
- $a_{k,\; i} = 1$ if $l_{i} = l_{k}$ and $w_{i} \leq d_{k}$
- $a_{k,\; i} = 1$ if $w_{i} = l_{k}$ and $l_{i} \leq d_{k}$ (rotated item)
- $a_{k,\; i} = 0$ otherwise

#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{i,\; k} \; \forall i\; \in \; P, \; \forall \; k \; \in \; M$
    - $y_{i,\; k} = 1$ if the demand at pickup vertex $i$ is loaded in stack $k$
    - $y_{i,\; k} = 0$ otherwise
- $u_{i}$ is the position of vertex $i$ in the route (order).
    - $0 \leq u_{i} \leq 2n$
    - $u_{0} = 0$ we start at the depot
- $s_{i,\; k}$ is the load of stack $k$ upon **leaving** vertex $i$
    - Domain $ : \{0, 1, .., q_{k}\}$ 
- $r_{i} \; \forall i\; \in \; P$
    - $r_{i} = 1$ if item $i$ is rotated
    - $r_{i} = 0$ otherwise
- $z_{i,\; k} \; \forall i\; \in \; P, \; \forall \; k \; \in \; M$
    - $z_{i,\; k} = 1$ if item $i$ is **rotated and assigned** to compartment $k$
    - $z_{i,\; k} = 0$ otherwise

#### Constraints
- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ (2)</center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ (3)</center>
    <br>
    
- The demand of every vertex is assigned to exactly one compartment. (Demand must be fulfilled)
    <br>
    <center> $\sum_{k \in M} y_{i, \; k} = 1$    $\; \forall i \in P$ (4)</center>
    <br>

- If $j$ is visited directly after $i$ i.e. $x_{i, \; j} = 1$, then $u_{j} \geq u_{i} + 1$
    <br>
    <center> $u_{j} \geq u_{i} + 1 - 2n(1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P \cup D$ (5)</center>
    <br>
    
- The item must be picked before it's delivered. Precedence constraint between pickups and deliveries.
    <br>
    <center> $u_{n + i} \geq u_{i} + 1 \; \forall i \in P$ (6)</center>
    <br>
    
- The state of the stack after loading and unloading.
    - The state of the assigned stack after picking is lowerbounded by the stack before picking and the picked (loaded) demand.
    
    <br>
    <center> $s_{j, \; k} \geq s_{i, \; k} + l_{j} * z_{j, \; k} + w_{j} * y_{j, \; k} - w_{j} * z_{j, \; k} - q_{k} * (1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (7)</center>
    <br>
    
    - The state of the assigned stack after delivering is lowerbounded by the stack before delivering and the (negative) delivered (unloaded) demand.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{i, \; k} - l_{j} * z_{j, \; k} - w_{j} * y_{j, \; k} + w_{j} * z_{j, \; k} - q_{k} * (1 - x_{i, \; (n + j)}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (8)</center>
    <br>
    
    - LIFO stacking constraint. The state of the stack after unloading is bounded by the state of the stack just after loading and the loaded / unloaded demand. (The state after unload $d{j}$ = state before loading $d{j}$ where $- d{j}$ is equivalent to $ + d_{j + n}$.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{j, \; k} - l_{j} * z_{j, \; k} - w_{j} * y_{j, \; k} + w_{j} * z_{j, \; k} - q_{k} * (1 - y_{j, \; k}) \; \forall j \in P, \; \forall k \in M$ (9)</center>
    <br>

- We always start at the depot.
    <br><center> $u_{0} = 0$ (10)</center><br>

- Vertices are order along the route.
    <br><center> $1 \leq u_{i}  \leq 2n \; \forall i \in P \cup D$ (11)</center><br>

- We always begin with empty stacks at the depot.
    <br><center> $s_{0, \; k} = 0 \; \forall k \in M$ (12)</center><br>

- The stack capacity is always maintained for all stacks. 
    <br><center> $0 \leq s_{i, \; k}  \leq q_{k} \; \forall i \in P \cup D \; \forall k \in M$ (13)</center><br>

- The definition of $z_{j, \; k}$ as item $j$ being rotated and assigned to compartment $k$. <br> Note: Needed for linearization of stack constraints.
    <br><center> $z_{i, \; k} = r_{i} * y_{i, \; k} \; \forall i \in P, \; \forall k \in M$</center><br> but the constraint is in non-linear so, we linearize by:
    <br><center> $z_{i, \; k} \leq r_{i} \; \forall i \in P, \; \forall k \in M$ (14)</center>
    <br><center> $z_{i, \; k} \leq y_{i, \; k} \; \forall i \in P, \; \forall k \in M$ (15)</center>
    <br><center> $z_{i, \; k} \geq r_{i} + y_{i, \; k} - 1 \; \forall i \in P, \; \forall k \in M$ (16)</center><br>

- Item-compartment compatibility constraints. 

    - An item can only be assigned to a compartment if the compartment can accommodate it. <br> Note: $a_{k, \; i}$ is predefined and this constraint can possibly be embedded implicitly in the domain for $y_{i, \; k}$, but for the sake of clarity, this is the explicit formulation.
    <br><center> $y_{i, \; k} \leq a_{k, \; i} \; \forall i \in P, \; \forall k \in M$ (17)</center><br>
    
    - If an item is assigned to a compartment of the same length, then the item must **not** be rotated.
    <br><center> $y_{i, \; k} \rightarrow \; \neg \; r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} = l_{k}$</center>
    <br><center> i.e. $y_{i, \; k} \leq 1 - r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} = l_{k}$ (18)</center><br>
    
    - If an item is assigned to a compartment of a different length, then the item must be rotated.
    <br><center> $y_{i, \; k} \rightarrow \; r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} \neq l_{k}$</center>
    <br><center> i.e. $y_{i, \; k} \leq r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} \neq l_{k}$ (19)</center><br>
    
    
#### Objective Function

- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j}$ (1)</center>
     
- Minimize the route cost and the number of rotated items.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j} + \sum_{i \in P} r_{i}$</center>

In [1]:
from gurobipy import *

In [2]:
import random 
random.seed(0)

#### Data 

- $G = (V, A)$
- $P = \{1, .., n\}$ $n$ pickup vertices for $n$ requests to be transported
- $I = \{(l_{i}, w_{i}), .. | \; i \; \in \; P\}$ $n$ 2D items representing the $n$ requests to be transported
- $D = \{n + 1, .., 2n\}$ $n$ delivery vertices for $n$ requests to be transported
- $V = \{0, 1, .., 2n\}$, where $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $M = \{1,.., m\}$ the set of Vehicle Compartments (stacks).
- $CompartmentDimensions = \{(l_{k}, d_{k}), .. |\; k \; \in \; M\}$
- $CompartmentCapacities = \{d_{k} |\; k \; \in \; M\}$ where $d_{k}$ is the *depth* of compartment $k$.

##### Inferred Data
$CompartmentCapabilities$ Matrix $MxI$
- $a_{k,\; i} = 1$ if item $i$ can be assigned to compartment $k$
- $a_{k,\; i} = 0$ otherwise
    
We say that $i$ is compatible with $k$ if it can fit in the compartment, considering both item orientations. In other words:
- $a_{k,\; i} = 1$ if $l_{i} = l_{k}$ and $w_{i} \leq d_{k}$
- $a_{k,\; i} = 1$ if $w_{i} = l_{k}$ and $l_{i} \leq d_{k}$ (rotated item)
- $a_{k,\; i} = 0$ otherwise

In [3]:
from dataclasses import dataclass

@dataclass
class Item:
    l: int
    w: int

@dataclass
class Compartment:
    l: int
    d: int

In [4]:
# Assume we have 9 items to be transported
nb_items = 9 # example 1
# First 4 items need to be in compartments [0: 3] 
# middle items can be in any compartments (rotatable)
# Last 3 items need to be in compartmetns [4, 5]
I = [None] + [Item(300, 400)] * 4 + [Item(600, 300)] + [Item(600, 300)] + [Item(600, 400)] * 3

# nb_items = 11 # example 2
# I = [None] + [Item(300, 400)] * 6 + [Item(600, 300)] + [Item(600, 300)] + [Item(600, 400)] * 3

# nb_items = 5 # example 3
# I = [None] + [Item(600, 300)] * 5

V = [i for i in range(nb_items * 2 + 1)]
P = V[1:nb_items + 1]
D = V[nb_items + 1:]
PUD = V[1:]
N = len(P) # == nb_items

# Assume an arbitrary graph
# Distance from / to depot is 0
random.seed(0)
A = [[random.randint(1, 10) if i != 0 and j !=0  and i != j else 0 for i in V] for j in V]

# Hack! Make pickup vertices further away from delivery vertices
same_cluster = lambda x, y: (x in P and y in P) or (x in D and y in D)
A = [[A[j][i] if same_cluster(i, j) else A[j][i] * 100 for i in V] for j in V]
C = A

# Assume we have 6 compartments in the vehicle, each of depth 800.
nb_compartments = 6
M = [i for i in range(nb_compartments)]
# 4 Compartments with dimensions (300, 800) and 2 with dimesnions (600, 800)
Compartments = [Compartment(300, 800)] * 4 + [Compartment(600, 800)] * 2
Capacities = [compartment.d for compartment in Compartments]

# Inferred Data

def item_fits_in_compartment(item: Item, compartment: Compartment) -> int:
    if item is None:
        return 0
    if item.l == compartment.l and item.w <= compartment.d:
        return 1
    if item.w == compartment.l and item.l <= compartment.d:
        return 1
    return 0

Capabilities = [[item_fits_in_compartment(i, c) for i in I] for c in Compartments]

In [5]:
#Capacities


In [6]:
# Declare and initialize the model
model = Model("1-PDPMS with 2D Items")

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-22


#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{i,\; k} \; \forall i\; \in \; P, \; \forall \; k \; \in \; M$
    - $y_{i,\; k} = 1$ if the demand at pickup vertex $i$ is loaded in stack $k$
    - $y_{i,\; k} = 0$ otherwise
- $u_{i}$ is the position of vertex $i$ in the route (order).
    - $0 \leq u_{i} \leq 2n$
    - $u_{0} = 0$ we start at the depot
- $s_{i,\; k}$ is the load of stack $k$ upon **leaving** vertex $i$
    - Domain $ : \{0, 1, .., q_{k}\}$ 
- $r_{i} \; \forall i\; \in \; P$
    - $r_{i} = 1$ if item $i$ is rotated
    - $r_{i} = 0$ otherwise
- $z_{i,\; k} \; \forall i\; \in \; P, \; \forall \; k \; \in \; M$
    - $z_{i,\; k} = 1$ if item $i$ is **rotated and assigned** to compartment $k$
    - $z_{i,\; k} = 0$ otherwise


In [7]:
x = model.addVars(V, V, vtype=GRB.BINARY, name="x")
y = model.addVars(P, M, vtype=GRB.BINARY, name="y")
u = model.addVars(V, vtype=GRB.INTEGER, name="u")
s = model.addVars(V, M, vtype=GRB.INTEGER, name="s")
r = model.addVars(P, vtype=GRB.BINARY, name="r")
z = model.addVars(P, M, vtype=GRB.BINARY, name="z")

In [8]:
# Set the bounds of the integer vars
for i in u:
    u[i].setAttr(GRB.Attr.LB, 0)    
    u[i].setAttr(GRB.Attr.UB, len(V) - 1)

for i, k in s:
    s[i, k].setAttr(GRB.Attr.LB, 0)    
    s[i, k].setAttr(GRB.Attr.UB, Capacities[k])
    
model.update()

### Constraints

#### Constraint
- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ (2)</center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ (3)</center>
    <br>
    

In [9]:
# 2
# Every vertex is left once
model.addConstrs((sum(x[i, j] for j in V) == 1 for i in V), name="out_vertex_constr")

# 3
# Every vertex is entered once
model.addConstrs((sum(x[j, i] for j in V) == 1 for i in V), name="in_vertex_constr")
model.update()

#### Constraint
- The demand of every vertex is assigned to exactly one compartment. (Demand must be fulfilled)
    <br>
    <center> $\sum_{k \in M} y_{i, \; k} = 1$    $\; \forall i \in P$ (4)</center>
    <br>

In [10]:
model.addConstrs((sum(y[i, k] for k in M) == 1 for i in P), name="fulfill_demands_constr")
model.update()

#### Constraint

- If $j$ is visited directly after $i$ i.e. $x_{i, \; j} = 1$, then $u_{j} \geq u_{i} + 1$
    <br>
    <center> $u_{j} \geq u_{i} + 1 - 2n(1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P \cup D$ (5)</center>
    <br>

In [11]:
model.addConstrs((u[j] >= u[i] + 1 - (2 * N * (1 - x[i, j])) for i in V for j in PUD), name="visiting_order_constr")
model.update()

#### Constraint

- The item must be picked before it's delivered. Precedence constraint between pickups and deliveries.
    <br>
    <center> $u_{n + i} \geq u_{i} + 1 \; \forall i \in P$ (6)</center>
    <br>

In [12]:
model.addConstrs((u[N + i] >= u[i] + 1 for i in P), name="pickup_before_delivery_constr")
model.update()

#### Constraint

- The state of the stack after loading and unloading.
    - The state of the assigned stack after picking is lowerbounded by the stack before picking and the picked (loaded) demand.
    
    <br>
    <center> $s_{j, \; k} \geq s_{i, \; k} + l_{j} * z_{j, \; k} + w_{j} * y_{j, \; k} - w_{j} * z_{j, \; k} - q_{k} * (1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (7)</center>
    <br>
    
    - The state of the assigned stack after delivering is lowerbounded by the stack before delivering and the (negative) delivered (unloaded) demand.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{i, \; k} - l_{j} * z_{j, \; k} - w_{j} * y_{j, \; k} + w_{j} * z_{j, \; k} - q_{k} * (1 - x_{i, \; (n + j)}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (8)</center>
    <br>
    
    - LIFO stacking constraint. The state of the stack after unloading is bounded by the state of the stack just after loading and the loaded / unloaded demand. (The state after unload $d{j}$ = state before loading $d{j}$ where $- d{j}$ is equivalent to $ + d_{j + n}$.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{j, \; k} - l_{j} * z_{j, \; k} - w_{j} * y_{j, \; k} + w_{j} * z_{j, \; k} - q_{k} * (1 - y_{j, \; k}) \; \forall j \in P, \; \forall k \in M$ (9)</center>
    <br>

In [13]:
# 7
load_expr = lambda j_, k_: I[j_].l * z[j_, k_] + I[j_].w * y[j_, k_] - I[j_].w * z[j_, k_]
model.addConstrs((s[j, k] >= s[i, k] + load_expr(j, k) - Capacities[k] * (1 - x[i, j]) for i in V for j in P for k in M), name="stack_after_pickup_constr")
# 
model.addConstrs((s[j, k] <= s[i, k] + load_expr(j, k) + Capacities[k] * (1 - x[i, j]) for i in V for j in P for k in M), name="stack_after_pickup_constr")


# 8
unload_expr = lambda j_, k_: -1 * load_expr(j_, k_)
# unload_expr = lambda j_, k_: - I[j_].l * z[j_, k_] - I[j_].w * y[j_, k_] + I[j_].w * z[j_, k_]
model.addConstrs((s[N + j, k] >= s[i, k] + unload_expr(j, k) - Capacities[k] * (1 - x[i, N + j]) for i in V for j in P for k in M), name="stack_after_delivery_constr")
#
model.addConstrs((s[N + j, k] <= s[i, k] + unload_expr(j, k) + Capacities[k] * (1 - x[i, N + j]) for i in V for j in P for k in M), name="stack_after_delivery_constr")

# 9
model.addConstrs((s[N + j, k] >= s[j, k] + unload_expr(j, k) - Capacities[k] * (1 - y[j, k]) for j in P for k in M), name="stack_LIFO_constr")
# Additional constraint
model.addConstrs((s[N + j, k] <= s[j, k] + unload_expr(j, k) + Capacities[k] * (1 - y[j, k]) for j in P for k in M), name="stack_LIFO_constr")

model.update()

In [14]:
# for i in P:
#     for k in M:
#         print(f"{I[i]} y[{i}, {k}]: {y[i, k].x} r[{i}]: {r[i].x} z[{i}, {k}]: {z[i, k].x}")

In [15]:
# for j in P:
#     k = next(k for k in M if y[j, k].x == 1)
#     i = next(i for i in V if x[i, j].x == 1)
#     i_ = next(i_ for i_ in V if x[i_, j + N].x == 1)
#     print(f"j: {j} from {i} to comp {k}")
# #     print(f"{s[j, k].x} == {s[i, k].x + I[j].l * r[j].x + I[j].w * (1 - r[j].x) }")
# #     print(f"s {s[i, k].x} expr {I[j].l * r[j].x + I[j].w * (1 - r[j].x) } l : {I[j].l} w: {I[j].w}: r: {r[j].x}")
# #     assert s[j, k].x == s[i, k].x + I[j].l * r[j].x + I[j].w * (1 - r[j].x) 
    
#     print(f"{s[N + j, k].x} == {s[i_, k].x - I[j].l * r[j].x - I[j].w * (1 - r[j].x) }")
#     print(f"s {s[i_, k].x} expr {I[j].l * r[j].x + I[j].w * (1 - r[j].x) } l : {I[j].l} w: {I[j].w}: r: {r[j].x}")
#     assert s[N + j, k].x >= s[i_, k].x - I[j].l * r[j].x - I[j].w * (1 - r[j].x)

#### Constraint

- We always start at the depot.
    <br><center> $u_{0} = 0$ (10)</center><br>

In [16]:
model.addConstr(u[0] == 0, name="start_at_depot_constr")
model.update()

#### Constraints

- Vertices are order along the route.
    <br><center> $1 \leq u_{i}  \leq 2n \; \forall i \in P \cup D$ (11)</center><br>

- We always begin with empty stacks at the depot.
    <br><center> $s_{0, \; k} = 0 \; \forall k \in M$ (12)</center><br>

- The stack capacity is always maintained for all stacks. 
    <br><center> $0 \leq s_{i, \; k}  \leq q_{k} \; \forall i \in P \cup D \; \forall k \in M$ (13)</center><br>

In [17]:
# 11
# constraint implicit in the domain of u

# 12
model.addConstrs((s[0, k] == 0 for k in M), name="empty_stacks_at_depot_constr")

# 13
# constraint implicit in the domain of s
model.update()

#### Constraints

- The definition of $z_{j, \; k}$ as item $j$ being rotated and assigned to compartment $k$. <br> Note: Needed for linearization of stack constraints.
    <br><center> $z_{i, \; k} = r_{i} * y_{i, \; k} \; \forall i \in P, \; \forall k \in M$</center><br> but the constraint is in non-linear so, we linearize by:
    <br><center> $z_{i, \; k} \leq r_{i} \; \forall i \in P, \; \forall k \in M$ (14)</center>
    <br><center> $z_{i, \; k} \leq y_{i, \; k} \; \forall i \in P, \; \forall k \in M$ (15)</center>
    <br><center> $z_{i, \; k} \geq r_{i} + y_{i, \; k} - 1 \; \forall i \in P, \; \forall k \in M$ (16)</center><br>

In [18]:
# 14
model.addConstrs((z[i, k] <= r[i] for i in P for k in M), name="z_r_constr")

# 15
model.addConstrs((z[i, k] <= y[i, k] for i in P for k in M), name="z_y_constr")

# 16
model.addConstrs((z[i, k] >= r[i] + y[i, k] - 1 for i in P for k in M), name="z_r_y_constr")
model.update()

#### Constraints

- Item-compartment compatibility constraints. 

    - An item can only be assigned to a compartment if the compartment can accommodate it. <br> Note: $a_{k, \; i}$ is predefined and this constraint can possibly be embedded implicitly in the domain for $y_{i, \; k}$, but for the sake of clarity, this is the explicit formulation.
    <br><center> $y_{i, \; k} \leq a_{k, \; i} \; \forall i \in P, \; \forall k \in M$ (17)</center><br>
    
    - If an item is assigned to a compartment of the same length, then the item must **not** be rotated.
    <br><center> $y_{i, \; k} \rightarrow \; \neg \; r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} = l_{k}$</center>
    <br><center> i.e. $y_{i, \; k} \leq 1 - r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} = l_{k}$ (18)</center><br>
    
    - If an item is assigned to a compartment of a different length, then the item must be rotated.
    <br><center> $y_{i, \; k} \rightarrow \; r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} \neq l_{k}$</center>
    <br><center> i.e. $y_{i, \; k} \leq r_{i} \; \forall i \in P, \; \forall k \in M, \; l_{i} \neq l_{k}$ (19)</center><br>

In [19]:
# 17
model.addConstrs((y[i, k] <= Capabilities[k][i] for i in P for k in M), name="item_comp_constr")

# 18
same_length = lambda i_, k_: I[i_].l == Compartments[k_].l
model.addConstrs((y[i, k] <= 1 - r[i] for i in P for k in M if same_length(i, k)), name="item_comp_not_rot_constr")

# 19
model.addConstrs((y[i, k] <= r[i] for i in P for k in M if not same_length(i, k)), name="item_comp_rot_constr")
model.update()

### Objective Function

- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j}$ (1)</center>
     
- Minimize the route cost and the number of rotated items.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j} + \sum_{i \in P} r_{i}$</center

In [20]:
# Set the objective function

# model.setObjective((sum(C[i][j] * x[i, j] for i in V for j in V)), GRB.MINIMIZE)

sum_expr = sum(C[i][j] * x[i, j] for i in V for j in V)
sum_expr += sum(r[i] for i in P)
model.setObjective(sum_expr, GRB.MINIMIZE)

In [21]:
model.write('1-PDPMS_2D_Items.lp')

In [22]:
# Run the optimization engine (implicitly runs model.update())
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4833 rows, 611 columns and 22635 nonzeros
Model fingerprint: 0x9d8c0fdb
Variable types: 0 continuous, 611 integer (478 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 754 rows and 129 columns
Presolve time: 0.07s
Presolved: 4079 rows, 482 columns, 14946 nonzeros
Variable types: 0 continuous, 482 integer (356 binary)
Found heuristic solution: objective 10400.000000

Root relaxation: objective 3.462500e+01, 312 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   34.62500    0   45 10400.0000   34.62500   100%     -    0s
     0     0   35.74419

In [23]:
def print_solution(optimized_model):
    for var in optimized_model.getVars():
        if abs(var.x) > 1e-6 or var.vtype != GRB.BINARY:
            print("{0}: {1}".format(var.varName, var.x))
    print("Total cost: {0}".format(optimized_model.objVal))
    return None

# display optimal values of decision variables
print_solution(model)  

x[0,5]: 1.0
x[1,4]: 1.0
x[2,8]: 1.0
x[3,7]: 1.0
x[4,6]: 1.0
x[5,9]: 1.0
x[6,3]: 1.0
x[7,2]: 1.0
x[8,11]: 1.0
x[9,1]: 1.0
x[10,12]: 1.0
x[11,15]: 1.0
x[12,17]: 1.0
x[13,18]: 1.0
x[14,0]: 1.0
x[15,16]: 1.0
x[16,13]: 1.0
x[17,14]: 1.0
x[18,10]: 1.0
y[1,0]: 1.0
y[2,2]: 1.0
y[3,1]: 1.0
y[4,2]: 1.0
y[5,5]: 1.0
y[6,3]: 1.0
y[7,4]: 1.0
y[8,5]: 1.0
y[9,4]: 1.0
u[0]: 0.0
u[1]: 3.0
u[2]: 8.0
u[3]: 6.0
u[4]: 4.0
u[5]: 1.0
u[6]: 5.0
u[7]: 7.0
u[8]: 9.0
u[9]: 2.0
u[10]: 15.0
u[11]: 10.0
u[12]: 16.0
u[13]: 13.0
u[14]: 18.0
u[15]: 11.0
u[16]: 12.0
u[17]: 17.0
u[18]: 14.0
s[0,0]: 0.0
s[0,1]: 0.0
s[0,2]: 0.0
s[0,3]: 0.0
s[0,4]: 0.0
s[0,5]: 0.0
s[1,0]: 400.0
s[1,1]: 0.0
s[1,2]: 0.0
s[1,3]: 0.0
s[1,4]: 400.0
s[1,5]: 300.0
s[2,0]: 400.0
s[2,1]: 400.0
s[2,2]: 800.0
s[2,3]: 600.0
s[2,4]: 800.0
s[2,5]: 300.0
s[3,0]: 400.0
s[3,1]: 400.0
s[3,2]: 400.0
s[3,3]: 600.0
s[3,4]: 400.0
s[3,5]: 300.0
s[4,0]: 400.0
s[4,1]: 0.0
s[4,2]: 400.0
s[4,3]: 0.0
s[4,4]: 400.0
s[4,5]: 300.0
s[5,0]: 0.0
s[5,1]: 0.0
s[5,2]: 0.0
s[5,

In [24]:
# The order of visiting the vertices
nexts = [j for i in V for j in V if x[i, j].x == 1]

# We always start at the depot
order = [0] 
curr = nexts[0]

# We always end at the depot
while (curr !=0):
    order.append(curr)
    curr = nexts[curr]
    
print(f"The order of visiting the vertices is {order[1:]}\n")
for i in order[1:]:
    action = "pickup" if i in P else "deliver"
    prop = "to" if i in P else "from"
    pick_idx = i if i in P else i - N
    comp = next(k for k in M if abs(y[pick_idx, k].x) > 1e-6)
    print(f"@ Vertex: {i} {action} item {pick_idx} {I[pick_idx]} {prop} {comp} rotated: {r[pick_idx].x} {[s[i, j].x for j in M]}")

The order of visiting the vertices is [5, 9, 1, 4, 6, 3, 7, 2, 8, 11, 15, 16, 13, 18, 10, 12, 17, 14]

@ Vertex: 5 pickup item 5 Item(l=600, w=300) to 5 rotated: 0.0 [0.0, 0.0, 0.0, 0.0, 0.0, 300.0]
@ Vertex: 9 pickup item 9 Item(l=600, w=400) to 4 rotated: 0.0 [0.0, 0.0, 0.0, 0.0, 400.0, 300.0]
@ Vertex: 1 pickup item 1 Item(l=300, w=400) to 0 rotated: 0.0 [400.0, 0.0, 0.0, 0.0, 400.0, 300.0]
@ Vertex: 4 pickup item 4 Item(l=300, w=400) to 2 rotated: 0.0 [400.0, 0.0, 400.0, 0.0, 400.0, 300.0]
@ Vertex: 6 pickup item 6 Item(l=600, w=300) to 3 rotated: 1.0 [400.0, 0.0, 400.0, 600.0, 400.0, 300.0]
@ Vertex: 3 pickup item 3 Item(l=300, w=400) to 1 rotated: 0.0 [400.0, 400.0, 400.0, 600.0, 400.0, 300.0]
@ Vertex: 7 pickup item 7 Item(l=600, w=400) to 4 rotated: 0.0 [400.0, 400.0, 400.0, 600.0, 800.0, 300.0]
@ Vertex: 2 pickup item 2 Item(l=300, w=400) to 2 rotated: 0.0 [400.0, 400.0, 800.0, 600.0, 800.0, 300.0]
@ Vertex: 8 pickup item 8 Item(l=600, w=400) to 5 rotated: 0.0 [400.0, 400.0, 8